In [1]:
import os
import tensorflow as tf
from transformers import TFGPT2Model

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer

import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
import re

In [2]:
# 0) 랜덤 시드 설정
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)
np.random.seed(SEED_NUM)

# 1) 사전학습된 토크나이저, 사전 설정
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token='<unused0>',
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

# 2) 데이터 불러오기
BATCH_SIZE = 32
NUM_EPOCHS = 3
SENT_MAX_LEN = 31

DATA_IN_PATH = './data_in/KOR'
DATA_OUT_PATH = "./data_out/KOR"

TRAIN_SNLI_DF = os.path.join(DATA_IN_PATH, 'KorNLI', 'snli_1.0_train.kor.tsv')
TRAIN_XNLI_DF = os.path.join(DATA_IN_PATH, 'KorNLI', 'multinli.train.ko.tsv')
DEV_XNLI_DF = os.path.join(DATA_IN_PATH, 'KorNLI', 'xnli.dev.ko.tsv')

train_data_snli = pd.read_csv(TRAIN_SNLI_DF, header=0, delimiter='\t', quoting=3)
train_data_xnli = pd.read_csv(TRAIN_XNLI_DF, header=0, delimiter='\t', quoting=3)
dev_data_xnli = pd.read_csv(DEV_XNLI_DF, header=0, delimiter='\t', quoting=3)

train_data_snli_xnli = train_data_snli.append(train_data_xnli)
train_data_snli_xnli = train_data_snli_xnli.dropna()
train_data_snli_xnli = train_data_snli_xnli.reset_index()

dev_data_xnli = dev_data_xnli.dropna()

print("Total # dataset: train - {}, dev - {}".format(len(train_data_snli_xnli), len(dev_data_xnli)))

# 3) 학습/검증 데이터 전처리
# 토크나이저를 제외하고는 5장에서 처리한 방식과 유사하게 접근
def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", " ", sent)
    return sent_clean

train_data_sents = []

for train_sent_1, train_sent_2 in train_data_snli_xnli[['sentence1', 'sentence2']].values:
    train_tokenized_sent_1 = vocab[tokenizer(clean_text(train_sent_1))]
    train_tokenized_sent_2 = vocab[tokenizer(clean_text(train_sent_2))]

    tokens = [vocab[vocab.bos_token]] 
    tokens += pad_sequences([train_tokenized_sent_1], 
                            SENT_MAX_LEN, 
                            value=vocab[vocab.padding_token], 
                            padding='post').tolist()[0] 
    tokens += [vocab[vocab.sep_token]]  
    tokens += pad_sequences([train_tokenized_sent_2], 
                            SENT_MAX_LEN, 
                            value=vocab[vocab.padding_token], 
                            padding='post').tolist()[0] 
    tokens += [vocab[vocab.eos_token]]

    train_data_sents.append(tokens)    

train_data_sents = np.array(train_data_sents, dtype=np.int64)

dev_data_sents = []

for dev_sent_1, dev_sent_2 in dev_data_xnli[['sentence1', 'sentence2']].values:
    dev_tokenized_sent_1 = vocab[tokenizer(clean_text(dev_sent_1))]
    dev_tokenized_sent_2 = vocab[tokenizer(clean_text(dev_sent_2))]

    tokens = [vocab[vocab.bos_token]] 
    tokens += pad_sequences([dev_tokenized_sent_1], 
                            SENT_MAX_LEN, 
                            value=vocab[vocab.padding_token], 
                            padding='post').tolist()[0] 
    tokens += [vocab[vocab.sep_token]]  
    tokens += pad_sequences([dev_tokenized_sent_2], 
                            SENT_MAX_LEN, 
                            value=vocab[vocab.padding_token], 
                            padding='post').tolist()[0] 
    tokens += [vocab[vocab.eos_token]]

    dev_data_sents.append(tokens)    

dev_data_sents = np.array(dev_data_sents, dtype=np.int64)

# 4) label 전처리 
# Label을 Netural, Contradiction, Entailment 에서 숫자 형으로 변경한다.
label_dict = {"entailment": 0, "contradiction": 1, "neutral": 2}

def convert_int(label):
    num_label = label_dict[label]    
    return num_label

train_data_snli_xnli["gold_label_int"] = train_data_snli_xnli["gold_label"].apply(convert_int)
train_data_labels = np.array(train_data_snli_xnli['gold_label_int'], dtype=int)

dev_data_xnli["gold_label_int"] = dev_data_xnli["gold_label"].apply(convert_int)
dev_data_labels = np.array(dev_data_xnli['gold_label_int'], dtype=int)

print("# train labels: {}, #dev labels: {}".format(len(train_data_labels), len(dev_data_labels)))

Total # dataset: train - 942808, dev - 2490
# train labels: 942808, #dev labels: 2490


In [3]:
class TFGPT2Classifier(tf.keras.Model):
    def __init__(self, dir_path, num_class):
        super(TFGPT2Classifier, self).__init__()
        
        self.gpt2 = TFGPT2Model.from_pretrained(dir_path)
        self.num_class = num_class
        
        self.dropout = tf.keras.layers.Dropout(self.gpt2.config.summary_first_dropout)
        self.classifier = tf.keras.layers.Dense(self.num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=self.gpt2.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs):
        outputs = self.gpt2(inputs)
        pooled_output = outputs[0][:, -1]
        # gpt2는 순방향 마스크 어텐션 기법을 쓰기 때문에
        # 마지막 토큰의 마지막 은닉상태값만 활용

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        return logits

In [6]:
BASE_MODEL_PATH = './gpt_ckpt'
sim_model = TFGPT2Classifier(dir_path=BASE_MODEL_PATH, num_class=3)

optimizer = tf.keras.optimizers.Adam(6.25e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
sim_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model_name = "tf2_gpt_kornli"

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=2)

checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

history = sim_model.fit(train_data_sents_short, train_data_labels_short, 
                        epochs=NUM_EPOCHS,
                        validation_data=(dev_data_sents_short, dev_data_labels_short),
                        batch_size=BATCH_SIZE, 
                        callbacks=[earlystop_callback, cp_callback])

All model checkpoint weights were used when initializing TFGPT2Model.

All the weights of TFGPT2Model were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFGPT2Model for predictions without further training.


./data_out/KOR\tf2_gpt_kornli -- Folder already exists 

Epoch 1/3
32/32 [==============================] - ETA: 0s - loss: 1.0992 - accuracy: 0.3460 
Epoch 1: val_accuracy improved from -inf to 0.33000, saving model to ./data_out/KOR\tf2_gpt_kornli\weights.h5
32/32 [==============================] - 490s 15s/step - loss: 1.0992 - accuracy: 0.3460 - val_loss: 1.0984 - val_accuracy: 0.3300
Epoch 2/3
32/32 [==============================] - ETA: 0s - loss: 1.0980 - accuracy: 0.3430 
Epoch 2: val_accuracy improved from 0.33000 to 0.37000, saving model to ./data_out/KOR\tf2_gpt_kornli\weights.h5
32/32 [==============================] - 557s 17s/step - loss: 1.0980 - accuracy: 0.3430 - val_loss: 1.0971 - val_accuracy: 0.3700
Epoch 3/3
32/32 [==============================] - ETA: 0s - loss: 1.1004 - accuracy: 0.3420 
Epoch 3: val_accuracy did not improve from 0.37000
32/32 [==============================] - 582s 18s/step - loss: 1.1004 - accuracy: 0.3420 - val_loss: 1.0957 - val_accuracy: 0

In [5]:
train_data_sents_short = train_data_sents[:1000]
train_data_labels_short = train_data_labels[:1000]

dev_data_sents_short = dev_data_sents[:100]
dev_data_labels_short = dev_data_labels[:100]

In [7]:
TEST_XNLI_DF = os.path.join(DATA_IN_PATH, 'KorNLI', 'xnli.test.ko.tsv')

test_data_xnli = pd.read_csv(TEST_XNLI_DF, header=0, delimiter='\t', quoting=3)
test_data_xnli = test_data_xnli[:100]

test_data_sents = []

for test_sent_1, test_sent_2 in test_data_xnli[['sentence1', 'sentence2']].values:
    test_tokenized_sent_1 = vocab[tokenizer(clean_text(test_sent_1))]
    test_tokenized_sent_2 = vocab[tokenizer(clean_text(test_sent_2))]

    tokens = [vocab[vocab.bos_token]] 
    tokens += pad_sequences([test_tokenized_sent_1], 
                            SENT_MAX_LEN, 
                            value=vocab[vocab.padding_token], 
                            padding='post').tolist()[0] 
    tokens += [vocab[vocab.sep_token]]  
    tokens += pad_sequences([test_tokenized_sent_2], 
                            SENT_MAX_LEN, 
                            value=vocab[vocab.padding_token], 
                            padding='post').tolist()[0] 
    tokens += [vocab[vocab.eos_token]]

    test_data_sents.append(tokens)    

test_data_sents = np.array(test_data_sents, dtype=np.int64)

test_data_xnli["gold_label_int"] = test_data_xnli["gold_label"].apply(convert_int)
test_data_labels = np.array(test_data_xnli['gold_label_int'], dtype=int)

print("# sents: {}, # labels: {}".format(len(test_data_sents), len(test_data_labels)))

sim_model.load_weights(checkpoint_path)

results = sim_model.evaluate(test_data_sents, test_data_labels, batch_size=1024)
print("test loss, test acc: ", results)

# sents: 100, # labels: 100
1/1 [==============================] - 6s 6s/step - loss: 1.0985 - accuracy: 0.3800
test loss, test acc:  [1.0984580516815186, 0.3799999952316284]
